In [ ]:
import numpy as np
import nibabel as nib
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams.update({'font.size': 13})

In [ ]:
# Load CNN RDMs 
depth = 15
cnnrdms = np.load('Sample_CNN_RDM.npz')['cnnrdms'] # This sample data was saved from '04_RSA_From_CNN_Perspective' 

In [ ]:
# Load sample ROI map saved from '05_RSA_From_Human_Perspective' 
num_cds = 5
cd_name_list = ['Digit', 'Object', 'Versus', 'Magnitude', 'Animacy']    
roi_masks = np.zeros((num_cds,64,76,64))
for cdi in range(num_cds):
    tmp = np.nan_to_num(nib.load(cd_name_list[cdi]+'.nii').get_fdata())
    roi_masks[cdi] = (tmp!=0)

In [ ]:
# Preparing plot
sns.set(style='white',font_scale=1.3)
col_arr = np.array(sns.color_palette('Spectral_r',depth)) # Color map
xticks = np.arange(depth)
xlabels=['1','2','3','4','5','6','7','8','9','10','11','12','13','Fc','Out\nput']

In [ ]:
# Define functions
def reorder_ani(rdm_org):
    rdm_reord = np.zeros((6,6))
    rdm_reord[:5,:5] = rdm_org[11:,11:] 
    rdm_reord[5,5] = rdm_org[10,10]
    rdm_reord[:5,5] = rdm_org[11:,10]
    rdm_reord[5,0:5] = rdm_org[10,11:]
    return rdm_reord
    
def rsa(stim_type,cnnrdm):
    if stim_type == 'Digit':
        iu = np.triu_indices(10,1)
        rho, p = stats.spearmanr(roirdm[:10,:10][iu],cnnrdm[:10,:10][iu])
    elif stim_type == 'Object':
        iu = np.triu_indices(6,1)
        rho, p = stats.spearmanr(roirdm[10:,10:][iu],cnnrdm[10:,10:][iu])
    elif stim_type == 'Versus':
        rho, p = stats.spearmanr(roirdm[:10,10:].flatten(),cnnrdm[:10,10:].flatten())
    elif stim_type == 'Magnitude': 
        rho, p = stats.spearmanr(roirdm[7:10,1:4].flatten(),cnnrdm[7:10,1:4].flatten())
    else: #stim_type == 'Animacy': 
        rho, p = stats.spearmanr(reorder_ani(roirdm)[3:,:3].flatten(),reorder_ani(cnnrdm)[3:,:3].flatten())
    return rho, p 

In [ ]:
# Load sample neural RDM made by '03_MVPA_Neural_RDMs.ipynb'
l = np.load('Sample_Neural_RDM.npz',allow_pickle=True)
brain_rdm = l['rdm']
brain_inform_list = l['info_list']

# Number of searchlight voxels
nvv = brain_rdm.shape[0]

# RSA
rsa_rslt = np.zeros((num_cds,depth,64,76,64))
for cdi in range(num_cds):
    stim_type = cd_name_list[cdi]
    roi_mask = roi_masks[cdi]
    for vi in range(nvv):
        idx = brain_inform_list[vi][1]
        # If the index is not at the ROI, continue.
        if roi_mask[idx] != 1:
            continue
        roirdm = brain_rdm[vi]
        for li in range(depth):
            rsa_rslt[cdi,li][idx], _ = rsa(stim_type,cnnrdms[li])

In [ ]:
# Bar graph : Average within ROI
# In this tutorial, you would use rho values rather than t-statistics from group data.
def save_bar_graph(cdi):
    # Here, assumed that there is one ROI
    roi_mask = roi_masks[cdi]
    fig, axes = plt.subplots(1,1,facecolor='white',constrained_layout=True)
    roi_idx = np.where(roi_mask== 1)
    vxlnum = roi_idx[0].shape[0]
    mean_ = np.zeros((depth))
    ste_ = np.zeros((depth))
    pos_info = np.zeros((depth))
    for li in range(depth):
        tmp = rsa_rslt[cdi,li].copy()
        vec = tmp[roi_idx]
        vec = vec[(vec>0)] # Use only positive values
        if np.sum(vec) > 0:
            pos_info[li] = 1
        mean_[li] = vec.mean()
        ste_[li] = vec.std()/vec.shape[0]
    pos_idx = np.where(pos_info==1)
    axes.set_title(cd_name_list[cdi]+"'s ROI")
    axes.bar(xticks[pos_idx],mean_[pos_idx],color=col_arr[pos_idx],width=0.9,linewidth=1.5,
                    yerr=ste_[pos_idx],error_kw=dict(lw=1.2),ecolor='k')
    axes.set_xticks(xticks)
    axes.set_xticklabels(xlabels)
    axes.set_xlabel('CNN layer')
    axes.set_ylabel('t-score')
    plt.show()
    plt.close()

In [ ]:
# Plot
for cdi in range(num_cds):
    save_bar_graph(cdi)